**Author:** Amparo Godoy Pastore <br>
**Date:** July 21, 2024
# Programming Project

## The Problem: Finding the Closest Pair of Points

> Given n points P1, P2, …, P1 in a plane, the task is to find the pair of points that is closest together. In this case we will measure the RT for the following input sizes: n=5000, n=10000, n=15000, n=20000, …, n=50000. More specifically, n takes 10 values from 5k to 50k with increment of 5k.

## Preliminaries and Helper Functions

In [19]:
import math
import numpy as np
import time

In [26]:
# HELPER FUNCTIONS
def get_distance(p1, p2):
    """Compute distance between two points."""
    return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

def generate_unique_points(n):
    """Generate n unique random points."""
    points = set()
    while len(points) < n:
        point = (np.random.randint(0, 100000), np.random.randint(0, 100000))
        points.add(point)
    return list(points)

def measure_runtime(algorithm, P):
    """Measure the runtime of the given algorithm with the provided points."""
    start_time = time.time()
    result = algorithm(P)
    end_time = time.time()
    return end_time - start_time

## Brute-Force Algorithm

In [27]:
def BruteForceClosestPoints(P):
    """
    Brute Force Algorithm for Closest Points Problem
    ------------------------------------------------
    Arguments:
    - P: a list of n points n >=2
    Returns:
    - Indexes of the two closest points
    """
    d_min = float('inf')
    pair = (None, None)
    n = len(P)
    for i in range(n):
        for j in range(i + 1, n):
            distance = get_distance(P[i], P[j])
            if distance < d_min:
                d_min = distance
                index_1, index_2 = i, j
    return index_1, index_2

## Divide-and-Conquer Algorithm

In [33]:
def DivideAndConqClosestPair(P):
    """
    Divide and Conquer Algorithm for Closest Point Problem
    ------------------------------------------------------
    Arguments:
    - P: a list of n points n >=2
    Returns:
    - (p0, p1): pair of closest points
    """
    # construct Px and Py 
    Px = sorted(P, key=lambda p: p[0])
    Py = sorted(P, key=lambda p: p[1])
    # recursion
    return DivideAndConqClosestPair_Rec(Px, Py)
    
def DivideAndConqClosestPair_Rec(Px, Py):
    """Recursive part of ALG2"""
    # base case: if P is less than or equal to 3 use brute force
    n = len(Px)
    if n <= 3:
        index1, index2 = BruteForceClosestPoints(Px)
        return (Px[index1], Px[index2])

    # divide: construct Qx, Qy, Rx, Ry
    mid = n // 2
    mid_x = Px[mid][0]

    Qx = Px[:mid]
    Rx = Px[mid:]
    Qy = []
    Ry = []

    for point in Py:
        if point[0] <= mid_x:
            Qy.append(point)
        else:
            Ry.append(point)
    
    # conquer: recursively find closest pair in Q and R
    (q0, q1) = DivideAndConqClosestPair_Rec(Qx, Qy)
    (r0, r1) = DivideAndConqClosestPair_Rec(Rx, Ry)
    
    # combine
    dq = get_distance(q0, q1)
    dr = get_distance(r0, r1)
    delta = min(dq, dr)

    x_star = Qx[-1][0] # max x-coordinate of a point in set Q

    Sy = []
    for p in Py:
        if x_star - delta <= p[0] <= x_star + delta:
            Sy.append(p)

    min_d = float('inf')
    pair = (None, None) # s and s'
    for i, s in enumerate(Sy):
        for j in range(1, 16):
            if i + j < len(Sy):
                next_point = Sy[i + j]
                d = get_distance(s, next_point)
                if d < min_d:
                    min_d = d
                    pair = (s, next_point)

    if min_d < delta:
        return (pair[0], pair[1])
    elif dq < dr:
        return (q0, q1)
    else:
        return (r0, r1)

## Test Case

In [52]:
points = generate_unique_points(1000)

cp_dc = DivideAndConqClosestPair(points)
print(f"The closest pair of points are: {cp_dc[0]} and {cp_dc[1]}")

i, j = BruteForceClosestPoints(points)
print(f"The closest pair of points are: {points[i]} and {points[j]}")

The closest pair of points are: (24752, 45398) and (24767, 45434)
The closest pair of points are: (24767, 45434) and (24752, 45398)


In [53]:
dc_time =  measure_runtime(DivideAndConqClosestPair, points)
bf_time =  measure_runtime(BruteForceClosestPoints, points)

print(f"RT Divide and Conquer: {dc_time}")
print(f"RT Brute Force: {bf_time}")

RT Divide and Conquer: 0.020483970642089844
RT Brute Force: 0.45816969871520996


## Experiments

In [25]:
def run_experiments():
    sizes = range(5000, 50001, 5000)  
    iterations = 10
    
    for n in sizes:
        total_dc_time = 0
        total_bf_time = 0

        for _ in range(iterations):
            points = generate_unique_points(n)

            # RT for D and C algorithm
            dc_time = measure_runtime(DivideAndConqClosestPair, points)
            total_dc_time += dc_time

            # RT for Brute Force algorithm
            bf_time = measure_runtime(BruteForceClosestPoints, points)
            total_bf_time += bf_time

        avg_dc_time = total_dc_time / iterations
        avg_bf_time = total_bf_time / iterations

        print(f"n = {n}:")
        print(f"  Average runtime for Divide and Conquer: {avg_dc_time:.4f} seconds")
        print(f"  Average runtime for Brute Force: {avg_bf_time:.4f} seconds")

run_experiments()

n = 5000:
  Average runtime for Divide and Conquer: 0.1170 seconds
  Average runtime for Brute Force: 8.0329 seconds
n = 10000:
  Average runtime for Divide and Conquer: 0.2420 seconds
  Average runtime for Brute Force: 28.9916 seconds
n = 15000:
  Average runtime for Divide and Conquer: 0.3661 seconds
  Average runtime for Brute Force: 62.2239 seconds
n = 20000:
  Average runtime for Divide and Conquer: 0.5167 seconds
  Average runtime for Brute Force: 112.6450 seconds
n = 25000:
  Average runtime for Divide and Conquer: 0.8353 seconds
  Average runtime for Brute Force: 206.2605 seconds
n = 30000:
  Average runtime for Divide and Conquer: 1.0079 seconds
  Average runtime for Brute Force: 327.8413 seconds
n = 35000:
  Average runtime for Divide and Conquer: 1.1992 seconds
  Average runtime for Brute Force: 448.5602 seconds
n = 40000:
  Average runtime for Divide and Conquer: 1.3723 seconds
  Average runtime for Brute Force: 583.7606 seconds
n = 45000:
  Average runtime for Divide and C